In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
dbutils.widgets.text("target_dataset", "VCM_DMT_PRD")
dbutils.widgets.text("target_table", "a_store_sku_daily")
dbutils.widgets.text("metadata_schema", "udp_wcm_metadata_dev")
dbutils.widgets.text(
    "dependency_table","""VCM_DWH_PRD.F_REVENUE_POS_B2B, VCM_DWH_PRD.F_SPECIAL_INV_MOVEMENT, VCM_DWH_PRD.F_TRANSACTION_LINE, VCM_DWH_PRD.F_TRANSACTION_FC, VCM_DWH_PRD.F_SALE_PROD, VCM_DWH_PRD.F_INVENTORY, VCM_DWH_PRD.F_COGS_90D"""
)

In [0]:
dbutils.widgets.text("field_calday", "CALDAY")
dbutils.widgets.text("field_id", "HASH_ID")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
environment = dbutils.widgets.get("environment")
target_dataset = dbutils.widgets.get("target_dataset")
target_table = dbutils.widgets.get("target_table")
metadata_schema = dbutils.widgets.get("metadata_schema")
dependency_table = dbutils.widgets.get("dependency_table")
dependency_table = [x.strip().upper() for x in dependency_table.split(",")]
dependency_table = "'" + "','".join(dependency_table) + "'"

print(f"environment: {environment}")
print(f"target_dataset: {target_dataset}")
print(f"target_table: {target_table}")
print(f"catalog_name: {catalog_name}")
print(f"metadata_schema: {metadata_schema}")
print(f"dependency_table: {dependency_table}")

In [0]:
field_calday = dbutils.widgets.get("field_calday")
field_id = dbutils.widgets.get("field_id")

print(f"field_calday: {field_calday}")
print(f"field_id: {field_id}")

In [0]:
%run "../common/common_etl_load"

In [0]:
spark.sql(
f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dmt.a_store_sku_daily 
(
  hash_id INT,
  calday DATE,
  store_id STRING,
  addr_numbr STRING,
  product_id STRING,
  revenue STRUCT<
    rev_amt DOUBLE,
    rev_amt_pos DOUBLE,
    cogs DOUBLE,
    cogs_pos DOUBLE,
    front_margin DOUBLE,
    front_margin_pos DOUBLE,
    rev_promo_amt DOUBLE,
    rev_vinid_amt DOUBLE,
    rev_promotion_discount DOUBLE,
    rev_promotion_discount_exclude_tax DOUBLE,
    m_vinid_earn DOUBLE,
    billcnt_total BIGINT,
    billcnt_vinid BIGINT,
    billcnt_promo BIGINT,
    base_sale_qty_pos_b2b DOUBLE,
    base_sale_qty DOUBLE,
    base_promotion_qty DOUBLE,
    base_coupon_qty DOUBLE,
    base_promotion_coupon_qty DOUBLE,
    base_sale_qty_sll DOUBLE,
    cogs_sll DOUBLE,
    rev_amt_sll DOUBLE
  >,

  stock STRUCT<
    shrinkage_val DOUBLE,
    shrinkage_qty DOUBLE,
    shrinkage_val_normal DOUBLE,
    shrinkage_qty_normal DOUBLE,
    shrinkage_val_noclaim DOUBLE,
    shrinkage_qty_noclaim DOUBLE,
    closing_stock_value DOUBLE,
    closing_stock_quantity DOUBLE,
    closing_stock_quantity_sloc_1000 DOUBLE,
    closing_stock_value_sloc_1000 DOUBLE,
    closing_stock_quantity_sloc_2000 DOUBLE,
    closing_stock_value_sloc_2000 DOUBLE,
    closing_stock_quantity_sloc_2000_3005 DOUBLE,
    closing_stock_value_sloc_2000_3005 DOUBLE,
    closing_stock_quantity_sloc_3005 DOUBLE,
    closing_stock_value_sloc_3005 DOUBLE,
    loss_val DOUBLE,
    loss_qty DOUBLE,
    promo_shrinkage_val DOUBLE,
    promo_shrinkage_qty DOUBLE
  >,

  sap_analysis STRUCT<
    sap_rev_amt DOUBLE,
    sap_shrinkage_val DOUBLE,
    sap_cogs DOUBLE
  >,

  unit_soh_value FLOAT
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
"""
)


In [0]:
if CALDAY_IN_STR != '':
	spark.sql(f"""
	CREATE OR REPLACE TEMP VIEW temp_a_store_sku_daily AS
	WITH
	cogs_sll AS (
	SELECT
		calday,
		store_id,
		product_id,
		cogs_per_unit
		FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_cogs 
		WHERE calday >= '2024-08-20'
	),
	transaction_sll AS (
	SELECT
		DISTINCT
		calday,
		plant AS store_id,
		rpa_tnr AS transaction_no
	FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vcdext01
	WHERE _bic_zc_fgroup = 'OGID'
	AND _bic_zc_fname = 'B2B'
	AND _bic_zc_fvalue LIKE 'B2B%'
	),
	revenue_sll AS (
		SELECT
		a.calday,
		a.store_id,
		a.product_id,
		SUM(a.base_sale_qty) AS base_sale_qty_sll,
		SUM(COALESCE(a.base_sale_qty * c.cogs_per_unit,0)) AS cogs_sll,
    SUM(
    a.SALE_AMOUNT - a.TAX_AMOUNT + 
    CASE 
        WHEN a.DISCOUNT_TYPE IN ('ZRTA', 'ZCRT') THEN a.DISCOUNT_VALUE 
        ELSE 0 
    END
    ) AS rev_amt_sll
		FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_sale_prod a
		INNER JOIN transaction_sll b 
		ON a.calday = b.calday AND a.store_id = b.store_id AND LTRIM(a.transaction_no, '0') = LTRIM(b.transaction_no, '0')
		LEFT JOIN cogs_sll c ON a.calday = c.calday AND a.store_id = c.store_id AND a.product_id = c.product_id
	WHERE
		a.calday IN ({CALDAY_IN_STR}) AND
		a.product_id IS NOT NULL
		GROUP BY 1,2,3
	),
	store_txn AS (
		SELECT
			calday,
			a.store_id,
			a.product_id,
			transaction_type,
			COUNT(DISTINCT transaction_id) AS m_txn_cnt,
			COUNT(DISTINCT CASE WHEN vinid_card_no IS NOT NULL THEN transaction_id END) AS m_txn_cnt_vinid,
			COUNT(DISTINCT CASE WHEN promotion_sale > 0 THEN transaction_id END) AS m_txn_cnt_promo
		FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_transaction_line a
		WHERE 
			a.calday IN ({CALDAY_IN_STR}) AND
			a.calday < CURRENT_DATE()
		GROUP BY 1,2,3,4
	),
 	bill_cnt AS (
		SELECT
			calday,
			a.store_id,
			a.product_id,
			SUM(b.bill_count * a.m_txn_cnt) AS billcnt_total,
			SUM(b.bill_count * a.m_txn_cnt_vinid) AS billcnt_vinid,
			SUM(b.bill_count * a.m_txn_cnt_promo) AS billcnt_promo
		FROM
			store_txn a
			INNER JOIN {catalog_name}.udp_wcm_silver_dim.transaction_type b ON a.transaction_type = b.transaction_type
		GROUP BY 1,2,3
	),
	store_rev AS (
	WITH
	cogs AS (
		SELECT
			calday,
			store_id,
			product_id,
			cogs_per_unit,
			cogs_per_unit_b2b
		FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_cogs
		WHERE calday IN ({CALDAY_IN_STR})
	),
	drwin_rev_amt_b2b AS (
		SELECT
			a.calday,
			a.store_id,
			a.product_id,
			IF(b.prop_value = 'Wholesale', 'Wholesale', 'Retail') source_bill,
			SUM(a.sale_amount - a.tax_amount + IF(discount_type IN ('ZRTA', 'ZCRT'), a.discount_value, 0)) AS rev_amt_b2b,
			SUM(IFNULL(a.base_sale_qty * cogs_per_unit, 0)) AS cogs
		FROM
			{catalog_name}.udp_wcm_gold_vcm_dwh.f_sale_prod a
			INNER JOIN {catalog_name}.udp_wcm_silver_realtime.pos_wcm_saleout_bill_props b ON LTRIM(a.transaction_no, '0') = b.receipt_no
			INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b1 ON a.store_id = b1.store_id
			LEFT JOIN cogs c ON a.calday = c.calday AND a.store_id = c.store_id AND a.product_id = c.product_id
		WHERE
			a.CALDAY IN ({CALDAY_IN_STR}) AND
			a.product_id != 'DUMMY' AND
			b.prop_name = 'DrWinSource' AND
			b1.business_unit = '1009'
		GROUP BY 1,2,3,4
	),
	sale_b2b AS (
		SELECT
			a.calday,
			CASE 
					WHEN e.store_id IS NOT NULL AND a.calday >= e.cutoff_date 
					THEN e.store_id 
					ELSE a.store_id 
			END AS store_id,
			a.product_id,
			SUM(
				CASE
					WHEN c.type != 'Y100' OR c.type IS NULL
					THEN a.m_rev_amt
					ELSE 0
				END
			) AS sap_rev_amt,
			SUM(
				CASE
					WHEN a.distr_chan = '60'
					THEN a.m_rev_amt
					ELSE 0
				END
			) AS rev_amt_b2b,
			SUM(
				CASE
					WHEN c.type != 'Y100' OR c.type IS NULL
					THEN a.m_cogs
					ELSE 0
				END
			) AS sap_cogs,
			SUM(a.m_cogs) AS cogs,
			SUM(
				CASE
					WHEN a.distr_chan != '60'
					THEN a.m_cogs
					ELSE 0
				END
			) AS cogs_pos,
			SUM(
				CASE
					WHEN a.distr_chan = '60'
					THEN a.base_q
					ELSE 0
				END
			) AS base_sale_qty_b2b,
			SUM(a.m_vinid_earn) AS m_vinid_earn
		FROM
			{catalog_name}.udp_wcm_gold_vcm_dwh.f_revenue_pos_b2b a
			LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_customer_master c ON a.customer_master_id = c.id
			LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_franchise_config e ON a.store_id = e.ref_fc_store
		WHERE
			a.CALDAY IN ({CALDAY_IN_STR}) AND
			a.calday < CURRENT_DATE()
			AND a.product_id != 'DUMMY'
		GROUP BY 1,2,3
	)
	SELECT
     a.*,
			CASE
				WHEN c.BUSINESS_UNIT = '1009' AND a.CALDAY >= '2024-03-06'
				THEN COALESCE(b.REV_AMT_B2B, 0)
				ELSE a.REV_AMT_B2B
			END AS REV_AMT_B2B,
			CASE
				WHEN c.BUSINESS_UNIT = '1009' AND a.CALDAY >= '2024-03-06'
				THEN COALESCE(b.COGS, 0) + COALESCE(b1.COGS, 0)
				ELSE a.cogs
			END AS cogs_final,
			CASE
				WHEN c.BUSINESS_UNIT = '1009' AND a.CALDAY >= '2024-03-06'
				THEN COALESCE(b1.COGS, 0)
				ELSE a.COGS_POS
			END AS cogs_pos_final
	FROM
		sale_b2b a
		LEFT JOIN drwin_rev_amt_b2b b ON a.calday = b.calday AND a.store_id = b.store_id AND a.product_id = b.product_id AND b.source_bill = 'Wholesale'
		LEFT JOIN drwin_rev_amt_b2b b1 ON a.calday = b1.calday AND a.store_id = b1.store_id AND a.product_id = b1.product_id AND b1.source_bill = 'Retail'
		LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store c ON a.store_id = c.store_id
	),
	sale_prod AS (
		SELECT
			a.calday,
			a.store_id,
			a.product_id,
			a.transaction_no,
			a.base_sale_qty,
			a.promotion_qty,
			CASE 
				WHEN MAX(CASE WHEN a.discount_type = 'ZICO' THEN 1 ELSE 0 END) 
						OVER (PARTITION BY a.transaction_no, a.store_id, a.product_id, a.calday) = 1
				THEN a.base_sale_qty
				ELSE 0
		  END AS base_coupon_qty,
			a.sale_amount,
			a.tax_amount,
			a.discount_type,
			a.discount_value,
			a.promotion_sale,
			a.vinid_card_no,
			a.sales_unit,
			COALESCE(b.tax_amount_by_product, 0) AS tax_amount_by_product,
			COALESCE(b.sale_amount_by_product, 0) AS sale_amount_by_product
		FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_sale_prod a
		LEFT JOIN (
			SELECT
			a1.calday,
			a1.product_id,
			SUM(a1.tax_amount) AS tax_amount_by_product,
			SUM(a1.sale_amount) AS sale_amount_by_product
			FROM
			{catalog_name}.udp_wcm_gold_vcm_dwh.f_sale_prod a1
			INNER JOIN
			{catalog_name}.udp_wcm_silver_dim.transaction_type b1
			USING
			(transaction_type)
			WHERE
			a1.calday IN ({CALDAY_IN_STR})
   		AND a1.calday < CURRENT_DATE()
			AND a1.product_id != 'DUMMY'
			AND b1.bill_count = 1
			GROUP BY 1, 2
		) b USING (calday, product_id)
		WHERE 
			a.calday IN ({CALDAY_IN_STR}) AND
			a.calday < CURRENT_DATE()
			AND a.product_id != 'DUMMY'
	),
	store_pos AS (
			SELECT
				a.calday,
				a.store_id,
				a.product_id,
				SUM(a.base_sale_qty) AS base_sale_qty,
				SUM(a.promotion_qty * c.numerator/c.denomintr) AS base_promotion_qty,
				SUM(a.base_coupon_qty) AS base_coupon_qty,
				SUM(IF(a.base_coupon_qty <> 0, a.base_coupon_qty, a.promotion_qty * c.numerator/c.denomintr)) AS base_promotion_coupon_qty,
				SUM(
					CASE
						WHEN (b.business_unit = '1009' AND a.calday >= '2024-03-06') 
						THEN IF(IFNULL(b1.prop_value, 'Retail') != 'Wholesale', a.sale_amount - a.tax_amount + IF(discount_type IN ('ZRTA', 'ZCRT'), a.discount_value, 0), 0)
						ELSE a.sale_amount - a.tax_amount + IF(discount_type IN ('ZRTA', 'ZCRT'), a.discount_value, 0)
					END
				) AS rev_amt_pos,
				SUM(CASE WHEN a.promotion_sale <> 0 THEN a.promotion_sale - a.tax_amount END) AS rev_promo_amt,
				SUM(CASE WHEN a.vinid_card_no IS NOT NULL THEN a.sale_amount - a.tax_amount END) AS rev_vinid_amt,
				SUM(IF(a.discount_type = 'ZPRM', a.discount_value, 0)) AS rev_promotion_discount,
				IF(SUM(a.tax_amount_by_product) > 0, SUM(a.tax_amount_by_product) / (SUM(a.sale_amount_by_product) - SUM(a.tax_amount_by_product)), 0) AS rev_vat_amount
			FROM
				sale_prod a
				LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b ON a.store_id = b.store_id
				LEFT JOIN {catalog_name}.udp_wcm_silver_realtime.pos_wcm_saleout_bill_props b1 ON LTRIM(a.transaction_no, '0') = b1.receipt_no AND b1.prop_name = 'DrWinSource'
				LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_unit_conv c ON a.product_id = c.material AND a.sales_unit = c.mat_unit
			WHERE
				a.calday IN ({CALDAY_IN_STR}) AND
				a.calday < CURRENT_DATE()
				AND a.product_id != 'DUMMY'
			GROUP BY 1,2,3
			UNION ALL
			SELECT
				a.calday,
				a.store_id,
				a.product_id,
				SUM(a.pos_quantity * c.numerator/c.denomintr) AS base_sale_qty,
				SUM(IFNULL(a.promo_quantity,0) * c.numerator/c.denomintr) AS base_promotion_qty,
				NULL base_coupon_qty,
				NULL base_promotion_coupon_qty,
				NULL rev_amt_pos,
				NULL rev_promo_amt,
				NULL rev_vinid_amt,
				NULL rev_promotion_discount,
				NULL rev_vat_amount
			FROM  {catalog_name}.udp_wcm_gold_vcm_dwh.f_transaction_fc a
			LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_unit_conv c ON a.product_id = c.material AND a.uom = c.mat_unit
			WHERE 
			a.calday IN ({CALDAY_IN_STR}) AND
			a.calday < CURRENT_DATE() 
			GROUP BY 1,2,3
	), 
    shrinkage AS (
			SELECT
				a.calday,
				CASE 
					WHEN e.store_id IS NOT NULL AND a.calday >= e.cutoff_date THEN e.store_id
					ELSE a.store_id
				END AS store_id,
				a.product_id,
				SUM(CASE 
					WHEN movetype NOT IN ('701', '702') 
					THEN a.movement_value 
					ELSE 0 
    		END) AS sap_shrinkage_val,
				SUM(
					CASE
						WHEN (a.reason_code IN (1,2,3,5,6) OR a.reason_code BETWEEN 201 AND 213)
							AND a.movetype NOT IN ('701','702')
						THEN a.movement_value ELSE 0
					END
				) AS shrinkage_val,
				SUM(
					CASE
						WHEN (a.reason_code IN (1,2,3,5,6) OR a.reason_code BETWEEN 201 AND 213)
							AND a.movetype NOT IN ('701','702')
						THEN a.movement_quantity ELSE 0
					END
				) AS shrinkage_qty,
				SUM(
					CASE
						WHEN (a.reason_code IN (1,2,3,6)
									OR a.reason_code BETWEEN 201 AND 209
									OR a.reason_code BETWEEN 212 AND 213)
							AND a.movetype NOT IN ('701','702')
						THEN a.movement_value ELSE 0
					END
				) AS shrinkage_val_normal,
				SUM(
					CASE
						WHEN (a.reason_code IN (1,2,3,6)
									OR a.reason_code BETWEEN 201 AND 209
									OR a.reason_code BETWEEN 212 AND 213)
							AND a.movetype NOT IN ('701','702')
						THEN a.movement_quantity ELSE 0
					END
				) AS shrinkage_qty_normal,
				SUM(
					CASE
						WHEN a.reason_code BETWEEN 210 AND 211
							AND a.movetype NOT IN ('701','702')
						THEN a.movement_value ELSE 0
					END
				) AS shrinkage_val_noclaim,
				SUM(
					CASE
						WHEN a.reason_code BETWEEN 210 AND 211
							AND a.movetype NOT IN ('701','702')
						THEN a.movement_quantity ELSE 0
					END
				) AS shrinkage_qty_noclaim,
				SUM(
					CASE
						WHEN a.movetype IN ('701','702')
						THEN CASE WHEN a.movetype = '701' THEN -1 * a.movement_value ELSE a.movement_value END
						ELSE 0
					END
				) AS loss_val,
				SUM(
					CASE
						WHEN a.movetype IN ('701','702')
						THEN a.movement_quantity ELSE 0
					END
				) AS loss_qty,
				SUM(
					CASE
						WHEN a.reason_code = 4 AND a.movetype IN ('551','552')
						THEN a.movement_value ELSE 0
					END
				) AS promo_shrinkage_val,
				SUM(
					CASE
						WHEN a.reason_code = 4 AND a.movetype IN ('551','552')
						THEN a.movement_quantity ELSE 0
					END
				) AS promo_shrinkage_qty
			FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_special_inv_movement a
			LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_franchise_config e ON a.store_id = e.ref_fc_store
			WHERE
				a.calday IN ({CALDAY_IN_STR}) AND
				a.calday < CURRENT_DATE()
				AND a.product_id != 'DUMMY'
				AND movetype IN ('551','552','701','702')
			GROUP BY 1,2,3
	),
	cogs_90d AS (
		SELECT
			calday,
			store_id,
			product_id,
			unit_soh_value
		FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_cogs_90d
	),
	inventory AS (
		SELECT
				a.calday,
				a.store_id,
				a.product_id,
				a.closing_stock_value,
				a.closing_stock_quantity
		FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory a
		WHERE
			a.calday IN ({CALDAY_IN_STR}) 
			AND a.calday < CURRENT_DATE()
			AND a.product_id != 'DUMMY'
	),
	inventory_sloc_1000 AS (
		SELECT
			a.calday,
			a.store_id,
			a.product_id,
			a.closing_stock_quantity AS closing_stock_quantity_sloc_1000
		FROM
			{catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory_sloc_1000 a
		WHERE
			a.calday IN ({CALDAY_IN_STR})
			AND a.calday < CURRENT_DATE()
			AND a.product_id != 'DUMMY'
	),
	inventory_sloc_2000 AS (
		SELECT
			a.calday,
			a.store_id,
			a.product_id,
			a.closing_stock_quantity AS closing_stock_quantity_sloc_2000
		FROM
			{catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory_sloc_2000 a
		WHERE
			a.calday IN ({CALDAY_IN_STR})
			AND a.calday < CURRENT_DATE()
			AND a.product_id != 'DUMMY'
	),
	inventory_sloc_2000_3005 AS (
		SELECT
			a.calday,
			a.store_id,
			a.product_id,
			a.closing_stock_quantity AS closing_stock_quantity_sloc_2000_3005
		FROM
			{catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory_sloc_2000_3005 a
		WHERE
			a.calday IN ({CALDAY_IN_STR})
			AND a.calday < CURRENT_DATE()
			AND a.product_id != 'DUMMY'
	),
	inventory_sloc_3005 AS (
		SELECT
			a.calday,
			a.store_id,
			a.product_id,
			a.closing_stock_quantity AS closing_stock_quantity_sloc_3005
		FROM
			{catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory_sloc_3005 a
		WHERE
			a.calday IN ({CALDAY_IN_STR})
			AND a.calday < CURRENT_DATE()
			AND a.product_id != 'DUMMY'
	),
	main AS (
	SELECT
		COALESCE(a.calday, b.calday, c.calday, d.calday, e.calday) AS calday,
		COALESCE(a.product_id, b.product_id, c.product_id, d.product_id, e.product_id) AS product_id,
		COALESCE(a.store_id, b.store_id, c.store_id, d.store_id, e.store_id) AS store_id,
		COALESCE(b.rev_amt_pos, 0) AS rev_amt,
			b.rev_amt_pos,
			a.sap_rev_amt,
			b.rev_promo_amt,
			b.rev_vinid_amt,
			b.rev_promotion_discount,
			b.rev_vat_amount,
			c.shrinkage_val,
			c.sap_shrinkage_val,
			c.shrinkage_qty,
			c.shrinkage_val_normal,
			c.shrinkage_qty_normal,
			c.shrinkage_val_noclaim,
			c.shrinkage_qty_noclaim,
			c.loss_val,
			c.loss_qty,
			c.promo_shrinkage_val,
			c.promo_shrinkage_qty,
			d.closing_stock_value,
			d.closing_stock_quantity,
			d1.closing_stock_quantity_sloc_1000,
		COALESCE(
		CASE 
			WHEN d.closing_stock_quantity = 0 THEN NULL 
			ELSE d.closing_stock_value / d.closing_stock_quantity 
		END * d1.closing_stock_quantity_sloc_1000,
		0
		) AS closing_stock_value_sloc_1000,
		d2.closing_stock_quantity_sloc_2000,
		COALESCE(
		CASE 
			WHEN d.closing_stock_quantity = 0 THEN NULL
			ELSE d.closing_stock_value / d.closing_stock_quantity
		END * d2.closing_stock_quantity_sloc_2000,
		0
		) AS closing_stock_value_sloc_2000,
		d3.closing_stock_quantity_sloc_2000_3005,
		COALESCE(
		CASE 
			WHEN d.closing_stock_quantity = 0 THEN NULL
			ELSE d.closing_stock_value / d.closing_stock_quantity
		END * d3.closing_stock_quantity_sloc_2000_3005,
		0
		) AS closing_stock_value_sloc_2000_3005,
		d4.closing_stock_quantity_sloc_3005,
		COALESCE(
		CASE 
			WHEN d.closing_stock_quantity = 0 THEN NULL
			ELSE d.closing_stock_value / d.closing_stock_quantity
		END * d4.closing_stock_quantity_sloc_3005,
		0
		) AS closing_stock_value_sloc_3005,
			a.cogs_final as cogs,
            a.cogs_pos_final as cogs_pos,
			a.sap_cogs,
			COALESCE(b.base_sale_qty, 0) + COALESCE(a.base_sale_qty_b2b, 0) AS base_sale_qty_pos_b2b,
			a.m_vinid_earn,
			e.billcnt_total,
			e.billcnt_vinid,
			e.billcnt_promo,
			b.base_promotion_qty,
			b.base_sale_qty,
			b.base_coupon_qty,
			b.base_promotion_coupon_qty
		FROM store_rev a
		FULL JOIN store_pos b ON
				a.calday = b.calday
				AND a.store_id = b.store_id
				AND a.product_id = b.product_id
		FULL JOIN shrinkage c ON
				a.calday = c.calday
				AND a.store_id = c.store_id
				AND a.product_id = c.product_id
		FULL JOIN inventory d ON
				a.calday = d.calday
				AND a.store_id = d.store_id
				AND a.product_id = d.product_id
		FULL JOIN inventory_sloc_1000 d1 ON
			d.calday = d1.calday
			AND d.store_id = d1.store_id
			AND d.product_id = d1.product_id
		FULL JOIN inventory_sloc_2000 d2 ON
			d.calday = d2.calday
			AND d.store_id = d2.store_id
			AND d.product_id = d2.product_id
		FULL JOIN inventory_sloc_2000_3005 d3 ON
			d.calday = d3.calday
			AND d.store_id = d3.store_id
			AND d.product_id = d3.product_id
		FULL JOIN inventory_sloc_3005 d4 ON
			d.calday = d4.calday
			AND d.store_id = d4.store_id
			AND d.product_id = d4.product_id
		FULL JOIN bill_cnt e ON
				a.calday = e.calday
				AND a.store_id = e.store_id
				AND a.product_id = e.product_id
	),
	main_group AS (
		SELECT
			farm_fingerprint(CONCAT(
				COALESCE(CAST(a.calday AS STRING), ''),
				COALESCE(CAST(a.store_id AS STRING), ''),
				COALESCE(CAST(a.product_id AS STRING), '')
			)) hash_id,
			a.calday,
			a.store_id,
			a.product_id,
			SUM(a.rev_amt) AS rev_amt,
			SUM(a.rev_amt_pos) AS rev_amt_pos,
			SUM(a.cogs) AS cogs,
			SUM(a.cogs_pos) AS cogs_pos,
			SUM(a.rev_amt) - SUM(a.cogs) AS front_margin,
			SUM(a.rev_amt_pos) - SUM(a.cogs_pos) AS front_margin_pos,
			SUM(a.rev_promo_amt) AS rev_promo_amt,
			SUM(a.rev_vinid_amt) AS rev_vinid_amt,
			SUM(a.rev_promotion_discount) AS rev_promotion_discount,
			ROUND(SUM(a.rev_promotion_discount / (1 + a.rev_vat_amount))) AS rev_promotion_discount_exclude_tax,
			SUM(a.m_vinid_earn) AS m_vinid_earn,
			SUM(a.billcnt_total) AS billcnt_total,
			SUM(a.billcnt_vinid) AS billcnt_vinid,
			SUM(a.billcnt_promo) AS billcnt_promo,
			SUM(a.base_sale_qty_pos_b2b) AS base_sale_qty_pos_b2b,
			SUM(a.base_sale_qty) AS base_sale_qty,
			SUM(a.base_promotion_qty) AS base_promotion_qty,
			SUM(a.base_coupon_qty) AS base_coupon_qty,
			SUM(a.base_promotion_coupon_qty) AS base_promotion_coupon_qty,
			SUM(a.shrinkage_val) AS shrinkage_val,
			SUM(a.shrinkage_qty) AS shrinkage_qty,
			SUM(a.shrinkage_val_normal) AS shrinkage_val_normal,
			SUM(a.shrinkage_qty_normal) AS shrinkage_qty_normal,
			SUM(a.shrinkage_val_noclaim) AS shrinkage_val_noclaim,
			SUM(a.shrinkage_qty_noclaim) AS shrinkage_qty_noclaim,
			SUM(a.loss_val) AS loss_val,
			SUM(a.loss_qty) AS loss_qty,
			SUM(a.promo_shrinkage_val) AS promo_shrinkage_val,
			SUM(a.promo_shrinkage_qty) AS promo_shrinkage_qty,
			SUM(a.closing_stock_value) AS closing_stock_value,
			SUM(a.closing_stock_quantity) AS closing_stock_quantity,
			SUM(a.closing_stock_quantity_sloc_1000) AS closing_stock_quantity_sloc_1000,
			SUM(a.closing_stock_value_sloc_1000) AS closing_stock_value_sloc_1000,
			SUM(a.closing_stock_quantity_sloc_2000) AS closing_stock_quantity_sloc_2000,
			SUM(a.closing_stock_value_sloc_2000) AS closing_stock_value_sloc_2000,
			SUM(a.closing_stock_quantity_sloc_2000_3005) AS closing_stock_quantity_sloc_2000_3005,
			SUM(a.closing_stock_value_sloc_2000_3005) AS closing_stock_value_sloc_2000_3005,
			SUM(a.closing_stock_quantity_sloc_3005) AS closing_stock_quantity_sloc_3005,
			SUM(a.closing_stock_value_sloc_3005) AS closing_stock_value_sloc_3005,
			SUM(sap_rev_amt) AS sap_rev_amt,
			SUM(sap_shrinkage_val) AS sap_shrinkage_val,
			NULL AS sap_cogs
		FROM main a
		GROUP BY a.calday, a.store_id, a.product_id
	), 
	add_rec AS (
		SELECT plant, addr_numbr FROM (
			SELECT *, ROW_NUMBER() OVER (PARTITION BY plant ORDER BY end_date DESC) AS rn
			FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_0plant
		) WHERE rn = 1
	)
	SELECT 
		main_group.hash_id,
		main_group.calday,
		main_group.store_id,
		COALESCE(add_rec.addr_numbr, fc.addr_numbr) AS addr_numbr,
		main_group.product_id,
		STRUCT(
			rev_amt, rev_amt_pos, cogs, cogs_pos, front_margin, front_margin_pos, 
			rev_promo_amt, rev_vinid_amt, rev_promotion_discount, 
			rev_promotion_discount_exclude_tax, m_vinid_earn, 
			billcnt_total, billcnt_vinid, billcnt_promo,
			base_sale_qty_pos_b2b, base_sale_qty, base_promotion_qty, 
			base_coupon_qty, base_promotion_coupon_qty,
			b.base_sale_qty_sll,
			b.cogs_sll,
			b.rev_amt_sll
		) AS revenue,
		STRUCT(
			shrinkage_val, shrinkage_qty, shrinkage_val_normal, shrinkage_qty_normal, 
			shrinkage_val_noclaim, shrinkage_qty_noclaim,
			closing_stock_value, closing_stock_quantity, 
			closing_stock_quantity_sloc_1000, closing_stock_value_sloc_1000,
			closing_stock_quantity_sloc_2000, closing_stock_value_sloc_2000, 
			closing_stock_quantity_sloc_2000_3005, closing_stock_value_sloc_2000_3005,
			closing_stock_quantity_sloc_3005, closing_stock_value_sloc_3005, 
			loss_val, loss_qty, promo_shrinkage_val, promo_shrinkage_qty
		) AS stock,
		STRUCT(sap_rev_amt, sap_shrinkage_val, sap_cogs) AS sap_analysis,
		cogs_data.unit_soh_value
	FROM main_group
	LEFT JOIN add_rec 
		ON main_group.store_id = add_rec.plant
	LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_franchise_config fc 
		ON main_group.store_id = fc.store_id
	LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.f_cogs_90d cogs_data
		ON main_group.calday = cogs_data.calday 
		AND main_group.store_id = cogs_data.store_id 
		AND main_group.product_id = cogs_data.product_id
	LEFT JOIN revenue_sll b 
		ON main_group.calday = b.calday 
		AND main_group.store_id = b.store_id 
		AND main_group.product_id = b.product_id;
	""")
else:
  print("Failed")

In [0]:
# Get the calday list string from temp table
if CALDAY_IN_STR != "":
    calday_query = """
    SELECT
        CONCAT("'", CONCAT_WS("','", COLLECT_SET(CAST(TO_DATE(calday, 'yyyy-MM-dd') AS STRING))), "'") AS calday_in_str
    FROM temp_a_store_sku_daily
    """

    calday_df = spark.sql(calday_query)
    calday_in_str = calday_df.collect()[0][0]

    # Only run DELETE if the string is non-empty
    if calday_in_str.strip("'"):  # checks if there's any date inside the quotes
        spark.sql(f"""
        DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dmt.a_store_sku_daily
        WHERE calday IN ({calday_in_str})
        """)
    else:
        print("No valid CALDAYs found to delete.") 

In [0]:
if CALDAY_IN_STR != "":
    spark.sql(f"""
    INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dmt.a_store_sku_daily
    SELECT * FROM temp_a_store_sku_daily
    """).display()

In [0]:
spark.sql(f"""DROP VIEW IF EXISTS temp_a_store_sku_daily""")

In [0]:
if CALDAY_IN_STR == '':
    dbutils.notebook.exit("No valid calday's found In common-etl to insert data into Table: ETL_LOG_DEPENDENCIES and ETL_DELTA_TABLE and ")

In [0]:
dbutils.notebook.exit("Success")